In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


In [13]:
from datasets import load_dataset
train_data = load_dataset('nsmc',split='train[:1%]')
val_data = load_dataset('nsmc',split='train[-1%:]')
test_data = load_dataset('nsmc',split='test[:5%]')

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)
Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


In [25]:
train_data

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 1500
})

# Tokenizer 불러오기

In [14]:
from transformers import BertForSequenceClassification, BertTokenizer
HUGGINGFACE_MODEL_PATH = "klue/bert-base"
model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=2)
config = model.config
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [15]:
def transform(data):
    return tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

In [16]:
train =train_data.map(transform, batched=True)
val = val_data.map(transform, batched=True)
test = test_data.map(transform, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-d5ef1ed553545efe.arrow


In [17]:
len(train)

1500

In [18]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/0326'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [19]:
from datasets import load_metric
metric = load_metric('accuracy')
#metric = load_metric('glue',accuracy')
def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [20]:
trainer = Trainer(
    model= model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=train,    # training dataset
    eval_dataset=val,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 1500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 564


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.364409,0.840000
2,No log,0.545471,0.846000
3,0.309200,0.663444,0.847333


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-500
Configuration saved in /aiffel/aiffel/0326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=564, training_loss=0.2972673033991604, metrics={'train_runtime': 561.0603, 'train_samples_per_second': 8.021, 'train_steps_per_second': 1.005, 'total_flos': 1183999749120000.0, 'train_loss': 0.2972673033991604, 'epoch': 3.0})

group_by_length (bool, optional, defaults to False

— Whether or not to group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient). Only useful if applying dynamic padding.

The main idea behind the 2 optimizations is to avoid as much unused computation as possible:

Dynamic padding: limit the number of added pad tokens to reach the length of the longest sequence of each mini batch instead of a fixed value set for the whole train set. Because the number of added tokens changes across mini batches, we call it “dynamic” padding ;

Uniform length batching: we push the logic further by generating batches made of similar length sequences, so we avoid extreme cases where most sequences in the mini batch are short and we are required to add lots of pad tokens to each of them because 1 sequence of the same mini batch is very long.

In [21]:
trainer.evaluate(test)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8


{'eval_loss': 0.614677369594574,
 'eval_accuracy': 0.8532,
 'eval_runtime': 164.3683,
 'eval_samples_per_second': 30.419,
 'eval_steps_per_second': 3.802,
 'epoch': 3.0}

# Bucketing 적용

In [22]:
from transformers import BertForSequenceClassification, BertTokenizer
HUGGINGFACE_MODEL_PATH = "klue/bert-base"
model = BertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=2)
config = model.config
tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/klue/bert-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transform

In [23]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    group_by_length = True,
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)
trainer = Trainer(
    model= model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    data_collator = data_collator,
    train_dataset=train,    # training dataset
    eval_dataset=val,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 1500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 564


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.364608,0.836000
2,No log,0.601462,0.838667
3,0.302600,0.690262,0.841333


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/0326/checkpoint-500
Configuration saved in /aiffel/aiffel/0326/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/0326/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=564, training_loss=0.29039256792541934, metrics={'train_runtime': 575.8021, 'train_samples_per_second': 7.815, 'train_steps_per_second': 0.98, 'total_flos': 1183999749120000.0, 'train_loss': 0.29039256792541934, 'epoch': 3.0})

In [24]:
trainer.evaluate(test)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8


{'eval_loss': 0.6244327425956726,
 'eval_accuracy': 0.8528,
 'eval_runtime': 164.2709,
 'eval_samples_per_second': 30.438,
 'eval_steps_per_second': 3.805,
 'epoch': 3.0}

## Bucketing 유무의 비교

Bucketing이란 시퀀스의 길이별로 정렬하고 버킷에 그룹화를 진행한다. 각 버킷 내의 시퀀스는 유사한 길이를 가지고 더 효율적인 배치와 최소화된 패딩을 가진다. HuggingFace의 DatacollatorWithPadding을 통해 진행한다.

동적 패딩이란 각 배치 내의 가장 긴 시퀀스의 길이에 맞게 시퀀스를 패딩하는 것이다. 이로 모델은 각 배치에 필요한 만큼의 패딩만 처리되어, 모델의 효율성을 증가 시킬수 있다.

- Without Bucketing :
    - 'train_runtime': 561.0603,
    - 'train_samples_per_second': 8.021,
    - 'train_steps_per_second': 1.005,
    - 'eval_accuracy': 0.8532,
    - 'eval_runtime': 164.3683,
    - 'eval_samples_per_second': 30.419,
    - 'eval_steps_per_second': 3.802,
- With Bucketing :
    - 'train_runtime': 575.8021,
    - 'train_samples_per_second': 7.815,
    - 'train_steps_per_second': 0.98,
    - 'eval_accuracy': 0.8528,
    - 'eval_runtime': 164.2709,
    - 'eval_samples_per_second': 30.438,
    - 'eval_steps_per_second': 3.805,